In [2]:
pip install fake-useragent

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.yandex-team.ru/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
from fake_useragent import UserAgent
import requests 


page_link = 'https://ling.tspu.edu.ru/archive.html'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content,'html.parser')

In [28]:
import time

In [41]:
articles = []

In [31]:
from tqdm import tqdm

In [42]:
for btn in tqdm(soup.find_all('input', {'class': 'issue_button btn'})):
    year, issue = btn['id'].split('_')[-2:]
    issue_link = f'https://ling.tspu.edu.ru/archive?year={year}&issue={issue}&format=html'
    issue_response = requests.get(issue_link, headers={'User-Agent': UserAgent().chrome})
    time.sleep(2)
    issue_soup = BeautifulSoup(issue_response.content,'html.parser')
    for article in issue_soup.find_all('tr', {'class': 'article'}):
        article_link = article.find('a')['href']
        article_response = requests.get(article_link, headers={'User-Agent': UserAgent().chrome})
        time.sleep(2)
        article_soup = BeautifulSoup(article_response.content,'html.parser')
        articles.append({
            'year': year,
            'title': article_soup.find('h1', {'class': 'title'}).text,
            'authors': article_soup.find('p', {'class': 'authors'}).text,
            'abstract': article_soup.find('p', {'class': 'anotation'}).text,
            'journal_title': 'Томский журнал лингвистических и антропологических исследований'
        })

100%|██████████| 43/43 [23:19<00:00, 32.54s/it]


In [46]:
import pandas as pd
pd.DataFrame(articles).to_csv('tspu.csv')